In [28]:
# needed imports
from flask import Flask, render_template, request, jsonify
from sqlmodel import SQLModel, Field, create_engine, Session
import uuid
import hashlib
from module import check_strategy_timeframe, list_strategy_folder, download_data, run_backtest, read_latest_backtest
from models import TradeData, BacktestData
from config import * 
# extra imports
from pprint import pprint
import pandas as pd
import sqlite3


In [29]:
# Create a connection to the SQLite database (it will create the file if it doesn't exist)
conn = sqlite3.connect('backtest.db')

In [30]:
unique_id = uuid.uuid4().hex[:16]
data_hash = "test_hash"

In [31]:
# Read latest backtest
backtest_dict = read_latest_backtest(results_dir=LOCAL_RESULTS_DIR, backtest_uuid=unique_id, backtest_hash=data_hash)
# pprint(backtest_data)

# Filter out fields with unsupported types
backtest_data = {k: v for k, v in backtest_dict.items() if not isinstance(v, (list, dict))}

# Filter out fields with unsupported types
skipped_fields = {k: v for k, v in backtest_dict.items() if isinstance(v, (list, dict))}

if skipped_fields:
    print("Skipped fields:", skipped_fields)


Skipped fields: {'pairlist': ['ATOM/USDT'], 'minimal_roi': {'0': 0.01}}


In [32]:
backtest_df = pd.DataFrame([backtest_data])
backtest_df

,total_trades,trade_count_long,trade_count_short,total_volume,avg_stake_amount,profit_mean,profit_median,profit_total,profit_total_long,profit_total_short,...,drawdown_start_ts,drawdown_end,drawdown_end_ts,max_drawdown_low,max_drawdown_high,csum_min,csum_max,backtest_strategy,backtest_uuid,backtest_hash
0,56,56,0,18073.59645,322.742794,-0.001576,0.00999,-0.031518,-0.031518,0.0,...,1.724526e+12,2024-10-26 02:00:00,1.729908e+12,-69.057236,16.1706,930.942764,1016.1706,ADXMomentum,813ebb2af7df40d7,test_hash


In [33]:
backtest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 91 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   total_trades                     1 non-null      int64  
 1   trade_count_long                 1 non-null      int64  
 2   trade_count_short                1 non-null      int64  
 3   total_volume                     1 non-null      float64
 4   avg_stake_amount                 1 non-null      float64
 5   profit_mean                      1 non-null      float64
 6   profit_median                    1 non-null      float64
 7   profit_total                     1 non-null      float64
 8   profit_total_long                1 non-null      float64
 9   profit_total_short               1 non-null      float64
 10  profit_total_abs                 1 non-null      float64
 11  profit_total_long_abs            1 non-null      float64
 12  profit_total_short_abs    

In [34]:

# Save the DataFrame to the database, specifying the table name
# backtest_df.to_sql('backtest_data', conn, if_exists='replace', index=False)
backtest_df.to_sql('backtest_data', conn, if_exists='append', index=False)

# Close the connection
conn.close()